In [2]:
import pandas as pd
import numpy as np

In [3]:
import pandas as pd
df = pd.read_pickle("dummy.pkl")
df = df.drop(columns=["scraped_time","search","interests"])
df = df.drop(columns=["url"])


__Personnal Info__

In [4]:
json_array= []
for i in range(len(df)):
    json_array.append(df["personal_info"][i])
store_list = []
for item in json_array:
    store_details = {"name":None,"location":None}
    store_details['name'] = item['name']
    store_details['location'] = item['location']
    store_list.append(store_details)
dtv=pd.DataFrame(store_list,index=None)
df=df.drop(columns="personal_info")
df = pd.concat([df, dtv], axis=1, sort=False)

__Accomplishments__

In [5]:
json_array= []

for i in range(len(df)):
    json_array.append(df["accomplishments"][i])

store_list = []

for item in json_array:
    store_details = {"courses":None,"certifications":None, "projects":None,"languages":None,"nbre_languages":None,"honors":None,"organizations":None,"nbre_projet":None}
    store_details['certifications'] = len(item['certifications'])
    store_details['courses'] = len(item['courses'])
    store_details['projects'] = item['projects']
    store_details['languages'] = item['languages']
    store_details['honors'] = len(item['honors'])
    store_details['organizations'] = len(item['organizations'])
    store_details['nbre_projet'] = len(item['projects'])
    store_details['nbre_languages'] = len(item['languages'])
    store_list.append(store_details)

dtv=pd.DataFrame(store_list,index=None)
dtv=dtv.drop(columns="courses")
dtv=dtv.drop(columns="projects")
dtv=dtv.drop(columns="languages")
df=df.drop(columns="accomplishments")
df = pd.concat([df, dtv], axis=1, sort=False)


#### Tenure

In [6]:
import datetime
from dateutil import relativedelta
import numpy as np
"""
Description:
    a dict shape which contains, as keys, the labels of the months as used in the date shape in LinkedIn, and their corresponding integer value [1 .. 12]
"""
date_dict = {'Jan' : 1,
            'Feb' : 2,
            'Mar' : 3,
            'Apr' : 4,
            'May' : 5,
            'Jun' : 6,
            'Jul' : 7,
            'Aug' : 8,
            'Sep' : 9,
            'Oct' : 10,
            'Nov' : 11,
            'Dec' : 12,
            'juin' : 6,
            'sept.' : 9,
            'mai' : 5,
            'mars' : 3,
            'févr.' : 2,
            'avr.' : 4,
            'juil.' : 7,
            'oct.' : 10,
            'août' : 8,
            'janv.' : 1,
            'nov.' : 11,
             'déc.' : 12
            }

def Tenure_Ratio_Transformation(ratio, x):
    """
    Description:
        returns the ratio value if x (an integer) is greater or equal to the ratio, and it returns the value of x if x is lower than the ratio
    parameters:
        ratio: an integer to compare with
        x: an integer. The target to compare and change its value
    returns:
        an integer: the result of the operation
    """
    if x >= ratio:
        return ratio
    else:
        return x
    
def is_Present(date_range):
    """
    Description:
        Checks if the candidate in currently working at the job
    parameters:
        date_range: the date range of the experience
    returns:
        True if present, False if not
    """
    if "Aujourd’hui" in date_range or "Present" in date_range:
        return True
    return False

def Calculate_Year_Month(date_range):
    """
    Description:
        Calculates the duration in months of a given date range
    parameters:
        date_range: shape of ['start month', 'start year', 'end month', 'end year']
    returns:
        duration of an experience in months
    """
    start_date=datetime.datetime(date_range[1],date_range[0],1)
    end_date=datetime.datetime(date_range[3],date_range[2],1)
    r = relativedelta.relativedelta(end_date, start_date)
    years=r.years
    months=r.months
    return (months + years*12)

def Calculate_Duration(person):
    """
    Description:
        Calculates the Duration of all the experiences of a person passed in parameter
    parameter: 
        person: a person's all experiences
    returns:
        present_durations: array shape of the duration of each present jobs
        past_duration: array shape of the duration of each past jobs
    """
    present_durations = []
    past_duration = []
    nb_trainings = 0
    nbr_present_jobs = 0
    experience_duration = 0
    for experience in person["jobs"]:
        if any(c in experience['title'] for c in ['stagiaire', 'Stagiaire', 'stage', 'Stage', 'STAGE', 'Trainee', 'trainee', 'Internship', 'internship']):
            nb_trainings = nb_trainings+1
        else:
            date_range = experience["date_range"]
            if date_range is None:
                return present_durations, past_duration
            date_range_liste = date_range.split()
            if len(date_range_liste) == 3:
                if is_Present(date_range_liste):
                    nbr_present_jobs += 1
                    if nbr_present_jobs <= 1 :
                        experience_duration = Calculate_Year_Month([1,int(date_range_liste[0]), 2, 2020])
                        present_durations.append(experience_duration)
                        previous_year = int(date_range_liste[0])
                        previous_month = 1
                    else:
                        experience_duration = Calculate_Year_Month([1,int(date_range_liste[0]), previous_month, previous_year])
                        past_duration.append(experience_duration)
                        previous_year = int(date_range_liste[0])
                        previous_month = 1
                else:
                    experience_duration = Calculate_Year_Month([1,int(date_range_liste[0]), 2, int(date_range_liste[2])])
                    past_duration.append(experience_duration)
                    previous_year = int(date_range_liste[0])
                    previous_month = 1
            elif len(date_range_liste) >= 4:
                if is_Present(date_range_liste):
                    nbr_present_jobs += 1
                    if nbr_present_jobs <= 1:
                        if date_range_liste[0] in date_dict.keys():
                            experience_duration = Calculate_Year_Month([date_dict[date_range_liste[0]],int(date_range_liste[1]), 2, 2020])
                        else:
                            experience_duration = Calculate_Year_Month([1,int(date_range_liste[0]), 2, 2020])
                        present_durations.append(experience_duration)
                        previous_year = int(date_range_liste[1])
                        previous_month = date_dict[date_range_liste[0]]
                    else:
                        if date_range_liste[0] in date_dict.keys():
                            experience_duration = Calculate_Year_Month([date_dict[date_range_liste[0]],int(date_range_liste[1]), previous_month, previous_year])
                        else:
                            experience_duration = Calculate_Year_Month([1,int(date_range_liste[0]), previous_month, previous_year])
                        past_duration.append(experience_duration)
                        previous_year = int(date_range_liste[1])
                        previous_month = date_dict[date_range_liste[0]]                      
                else:
                    if date_range_liste[0] in date_dict.keys():
                        if date_range_liste[3] in date_dict.keys():
                            experience_duration = Calculate_Year_Month([date_dict[date_range_liste[0]],int(date_range_liste[1]), date_dict[date_range_liste[3]], int(date_range_liste[4])])
                        else:
                            experience_duration = Calculate_Year_Month([date_dict[date_range_liste[0]],int(date_range_liste[1]), 1, int(date_range_liste[3])])

                        previous_year = int(date_range_liste[1])
                        previous_month = date_dict[date_range_liste[0]]
                    else:
                        experience_duration = Calculate_Year_Month([1,int(date_range_liste[0]), date_dict[date_range_liste[2]], int(date_range_liste[3])])
                        previous_year = int(date_range_liste[0])
                        previous_month = 1
                    past_duration.append(experience_duration)
                    
    return present_durations, past_duration

def calculate_Tenure(present_durations, past_duration, ratio):
    """
    Description:
        Calculates the probability of a candidate to stay at a job post for a minimum period passed in parameter (the ratio)
    parameters:
        present_durations: array shape of the duration of each present jobs
        past_duration: array shape of the duration of each past jobs
        ratio: the requested minimum duration in months
    returns:
        the Tenure in percentage [0, 1]
    """
    past_duration_sigmoid = []
    present_durations_sigmoid = []
    for d in past_duration:
        past_duration_sigmoid.append(Tenure_Ratio_Transformation(ratio, d))
    for d in present_durations:
        present_durations_sigmoid.append(Tenure_Ratio_Transformation(ratio, d))
    
    if len(present_durations_sigmoid) == 0:
        present_durations_sigmoid.append(0)
    if len(past_duration_sigmoid) == 0:
        past_duration_sigmoid.append(0)
    present_array = np.array(present_durations_sigmoid)
    past_array =  np.array(past_duration_sigmoid)
    
    return round(((0.7 * sum(past_array / ratio) / len(past_array)) + (0.3 * (present_array / ratio)))[0],2)

df_tenure = pd.DataFrame(columns=['Tenure']) #Creating an empty pandas dataframe for the tenure, this will be later concatenated with the intial dataframe

i=0
for person in df.experiences:
    pres, past = Calculate_Duration(person)
    df_tenure = df_tenure.append([{'Tenure' : calculate_Tenure(pres,past, 36)}],ignore_index=True)

df_final = pd.concat([df,df_tenure], axis=1)

In [7]:
df = df_final

# Skills

In [8]:
def dic_split(arr):
    skills=[]
    score=[]
    for i in arr:
        skills.append(i["name"])
        score.append(i["endorsements"])
    return [skills,score]
df["skills"] = pd.DataFrame(df["skills"]).applymap(lambda x : dic_split(x))
import pickle
with open('skills_dict',"rb") as handle:
    dict = pickle.load(handle)
dict["Nones"]=["Nones"]
def transf(skill,dic):
    t = 0
    for d in dic.keys():
        if skill in dic[d][0]:
                 t = 1
                 a = d
    if t == 1:
        return a
    else:
        return None
for j in range(len(df)):
    s = df["skills"][j][0]
    for i in range(len(s)):
        s[i] =transf(s[i],dict)
l=[]
for i in range(len(df)):
    for j in range(len(df["skills"][i][0])):
        l.append(df["skills"][i][0][j])
        l=list(set(l))
import random
def rem_none(arr) :
    new_arr = [[],[]]
    for i in range(len(arr[0])):
        if arr[0][i]!=None:
            new_arr[0].append(arr[0][i])
            new_arr[1].append(random.randrange(3)+1)
    return new_arr
for i in range(len(df)):
    df["skills"][i] = rem_none(df["skills"][i])
to_remove=[]
for i in range(len(df)):
    if len(df["skills"][i][0]) == 0 :
        to_remove.append(i)
df=df.drop(df.index[to_remove])
df = df.reset_index()
df = df.drop(columns="index")
df = df.drop(columns="experiences")

C:\Users\Brahim\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [11]:
df

,skills,name,location,certifications,nbre_languages,honors,organizations,nbre_projet,Tenure
0,"[[HTML, PHP, MySQL, Drupal], [2, 2, 2, 3]]",**************,Tunisie,0,3,0,0,5,0.33
1,"[[MySQL, HTML, C++, PHP], [1, 3, 2, 3]]",**************,Tunisie,0,1,0,0,4,0.32
2,"[[CSS, HTML, MySQL, Drupal, CMS, SASS], [1, 1,...",**************,"Nabeul Governorate, Tunisia",0,3,0,0,0,0.52
3,"[[PHP, CSS, HTML, MySQL, GIT, Symfony, C++, SV...",**************,"Gouvernorat de l’Ariana, Tunisia",0,3,0,0,4,0.38
4,"[[MySQL, MongoDB, Scrum, C++], [3, 1, 1, 3]]",**************,France,0,4,2,0,2,0.39
...,...,...,...,...,...,...,...,...,...
6248,"[[java, PHP], [3, 3]]",**************,"Sfax Médina, Gouvernorat de Sfax, Tunisia",0,0,0,0,0,0.08
6249,"[[C++], [3]]",**************,"Gouvernorat de l’Ariana, Tunisia",0,3,0,0,0,0.39
6250,"[[MongoDB, C++, CSS], [1, 1, 2]]",**************,"Paris Area, France",0,2,0,0,0,0.38
6251,"[[C++, MySQL], [2, 2]]",**************,"Gouvernorat de Bizerte, Tunisia",0,0,0,0,0,0.10
